In [1]:
# Import dependancies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import requests
import csv
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import datetime as dt
import matplotlib.pyplot as plt

### Get Vaccine Data from CDC

In [2]:
# Get Vaccinations Table from CDC website
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit("https://covid.cdc.gov/covid-data-tracker/#vaccinations")
vaccine_html = browser.html
browser.click_link_by_id("vaccinations-table-toggle")
cdc_html = browser.html
cdc_parsed = bs(cdc_html, "html.parser")
table = cdc_parsed.select("table")
browser.quit()

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/88.0.4324.96/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\coled\.wdm\drivers\chromedriver\win32\88.0.4324.96]


In [3]:
# Read the CDC Vaccine HTML Table
vaccines_df = pd.read_html(str(table))[0]

In [4]:
# Change "New York State" to "New York" for data merging
vaccines_df.loc[vaccines_df["State/Territory/Federal Entity"] == "New York State", "State/Territory/Federal Entity"] = "New York"

In [5]:
# Rename "State/Territory/Federal Entity" column to "state" for data merging
vaccines_df = vaccines_df.rename(columns={"State/Territory/Federal Entity": "state"})

In [6]:
# Overwrite Latest Vaccine CSV
vaccines_df.to_csv("../data/cdc-vaccine-data.csv")

### NYT / Rearc COVID Data

In [7]:
# Read Daily COVID-19 CSV from AWS S3 Bucket - Rearc / NY Times Data 
us_covid_cases_data = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-states/us-states.csv", stream=True)

In [8]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/us-covid-data.csv",  "wb") as file:
    file.write(us_covid_cases_data.content)

In [9]:
# Read US COVID Data CSV for data merging and cleaning 
us_covid_cases_df = pd.read_csv("../data/us-covid-data.csv")

In [23]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20249,2021-03-05,Virginia,51,583060,9428
20250,2021-03-05,Washington,53,346733,5104
20251,2021-03-05,West Virginia,54,132964,2318
20252,2021-03-05,Wisconsin,55,620663,7099


# State Overview Data

In [24]:
# Get yesterday's date, which is the latest data 
today = dt.date.today()
yesterday = today - dt.timedelta(days = 1)
latest_data = yesterday.strftime("%Y-%m-%d")

In [25]:
# Get latest total cases by state for latest date
us_total_cases_to_date_df = us_covid_cases_df.groupby(["date", "state"]).sum()
us_total_cases_to_date_df.reset_index(inplace=True)
us_total_cases_to_date_df = us_total_cases_to_date_df.loc[us_total_cases_to_date_df["date"] == latest_data]

In [26]:
# Read State Lat-Lon CSV and rename columns for easier cleaning and merging
state_latlons_df = pd.read_csv("../data/statelatlong.csv")
state_latlons_df.rename(columns={"State": "abbr", "Latitude": "lat", "Longitude": "lon", "City": "state"}, inplace=True)

In [27]:
# Read State Population CSV for cleaning and merging
state_pop_df = pd.read_csv("../data/state_populations_cleaned.csv")

In [28]:
# Merge, rename, and focus on defined columns
state_overview_master_df = pd.merge(state_pop_df, state_latlons_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["state", "abbr", "lat", "lon", "population"]]

In [29]:
# Merge with us_total_cases_to_date_df and focus on defined columns
state_overview_master_df = pd.merge(state_overview_master_df, us_total_cases_to_date_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths"]]

In [30]:
# Merge with Vaccination data, rename columns, focus data columns
state_overview_master_df = pd.merge(state_overview_master_df, vaccines_df, how="inner", on="state")
state_overview_master_df = state_overview_master_df.rename(columns={"Total Doses Administered by State where Administered": "total_doses_administered", "Doses Administered per 100k by State where Administered": "doses_administered_per_100k"})
state_overview_master_df = state_overview_master_df[["date", "state", "abbr", "lat", "lon", "population", "fips", "cases", "deaths", "total_doses_administered", "doses_administered_per_100k"]]

In [31]:
# Add Empty Column for New Calculation
state_overview_master_df["percent_vaccinated"] = ""

In [32]:
# Calcuation for percent vaccinated by state
for index, row in state_overview_master_df.iterrows():
    population = row["population"]
    vaccinated = row["total_doses_administered"]
    percent_vaccinated = round((vaccinated / population) * 100, 2)
    
    state_overview_master_df.at[index, "percent_vaccinated"] = percent_vaccinated

In [33]:
# Convert percent vaccinated into a float
state_overview_master_df = state_overview_master_df.astype({"percent_vaccinated": 'float64'})

In [34]:
state_overview_master_df.to_csv("../data/state-master-data.csv")

In [35]:
state_overview_master_df

,date,state,abbr,lat,lon,population,fips,cases,deaths,total_doses_administered,doses_administered_per_100k,percent_vaccinated
0,2021-03-05,Alabama,AL,32.601011,-86.680736,4903185,1,498887,10122,1099697,22428.0,22.43
1,2021-03-05,Alaska,AK,61.302501,-158.775020,731545,2,58870,292,288625,39454.0,39.45
2,2021-03-05,Arizona,AZ,34.168219,-111.930907,7278717,4,823384,16272,2043739,28078.0,28.08
3,2021-03-05,Arkansas,AR,34.751928,-92.131378,3017804,5,324326,5283,744139,24658.0,24.66
4,2021-03-05,California,CA,37.271875,-119.270415,39512223,6,3592219,53838,10415023,26359.0,26.36
5,2021-03-05,Colorado,CO,38.997934,-105.550567,5758736,8,436848,6075,1580162,27439.0,27.44
6,2021-03-05,Connecticut,CT,41.518784,-72.757507,3565287,9,285330,7704,1157114,32455.0,32.46
7,2021-03-05,Delaware,DE,39.145251,-75.418921,973764,10,87874,1453,264601,27173.0,27.17
8,2021-03-05,District of Columbia,DC,38.899349,-77.014567,705749,11,41122,1027,220634,31262.0,31.26
9,2021-03-05,Florida,FL,27.975728,-83.833017,21477737,12,1936199,31521,5688357,26485.0,26.48


# County Overview Data

In [36]:
populations_county = pd.read_csv("../data/census-bureau-population-by-county.csv")
poverty_county = pd.read_csv("../data/poverty-and-median-household-income-data-by-us-county-2019.csv")
state_keys = pd.read_csv("../data/state-names-codes.csv")

In [37]:
poverty_county = poverty_county[["Postal Code", "county", "Poverty Estimate, All Ages", "Median Household Income"]]
poverty_county = poverty_county.rename(columns={"Postal Code": "code", "Poverty Estimate, All Ages": "poverty_est_all_ages", 
                                              "Median Household Income": "median_household_income"})
poverty_county["county"] = poverty_county["county"].str.rstrip()

In [38]:
state_keys = state_keys.rename(columns={"Code": "code", "State": "state"})
state_keys = state_keys[["state", "code"]]

In [39]:
poverty_county = pd.merge(poverty_county, state_keys, how="inner", on="code")
poverty_county.drop(columns={"state"}, inplace=True)

In [40]:
populations_county = populations_county.merge(state_keys, how="inner", on="state")
populations_county = populations_county[["code", "county", "state", "2019_population_est"]]
populations_county["code"] = populations_county["code"].str.rstrip()
populations_county["county"] = populations_county["county"].str.rstrip()

In [41]:
merged_data = populations_county.merge(poverty_county, how='inner', left_on=["code", "county"], right_on=["code", "county"])

In [42]:
merged_data.dropna(how="any", inplace=True)

### Retrieve County Cases Data

In [43]:
county_cases = requests.get("https://covid19-lake.s3.us-east-2.amazonaws.com/rearc-covid-19-nyt-data-in-usa/csv/us-counties/us-counties.csv", stream=True)

In [44]:
# Overwrite the US COVID Data CSV with the latest one 
with open("../data/county-covid-data.csv",  "wb") as file:
    file.write(county_cases.content)

In [45]:
# Read US COVID Data CSV for data merging and cleaning 
us_county_cases = pd.read_csv("../data/county-covid-data.csv")

In [46]:
us_county_cases

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1092447,2021-03-05,Sweetwater,Wyoming,56037.0,3829,36.0
1092448,2021-03-05,Teton,Wyoming,56039.0,3375,9.0
1092449,2021-03-05,Uinta,Wyoming,56041.0,2070,12.0
1092450,2021-03-05,Washakie,Wyoming,56043.0,886,26.0


In [47]:
us_county_cases = us_county_cases.groupby(["date", "state", "county"]).sum()
us_county_cases.reset_index(inplace=True)

In [48]:
us_county_cases

,date,state,county,fips,cases,deaths
0,2020-01-21,Washington,Snohomish,53061.0,1,0.0
1,2020-01-22,Washington,Snohomish,53061.0,1,0.0
2,2020-01-23,Washington,Snohomish,53061.0,1,0.0
3,2020-01-24,Illinois,Cook,17031.0,1,0.0
4,2020-01-24,Washington,Snohomish,53061.0,1,0.0
...,...,...,...,...,...,...
1092447,2021-03-05,Wyoming,Sweetwater,56037.0,3829,36.0
1092448,2021-03-05,Wyoming,Teton,56039.0,3375,9.0
1092449,2021-03-05,Wyoming,Uinta,56041.0,2070,12.0
1092450,2021-03-05,Wyoming,Washakie,56043.0,886,26.0


In [49]:
us_county_cases = us_county_cases.merge(merged_data, how="left")

In [50]:
us_county_cases.to_csv("../data/county-cases-daily-master.csv")

In [51]:
latest_covid_cases = us_county_cases.loc[us_county_cases["date"] == latest_data]

In [52]:
latest_covid_cases.dropna(how="any", inplace=True)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [53]:
# latest_covid_cases["poverty_est_all_ages"] = int(row["poverty_est_all_ages"].replace(',', ""))

for index, row in latest_covid_cases.iterrows():
    latest_covid_cases.at[index, "poverty_est_all_ages"] = row["poverty_est_all_ages"].replace(',', "")
    latest_covid_cases.at[index, "median_household_income"] = row["median_household_income"].replace(',', "")
    
#  a = '1,000,000'
# >>> int(a.replace(',', ''))

In [54]:
for index, row in latest_covid_cases.iterrows():
    latest_covid_cases.at[index, "poverty_est_all_ages"] = row["poverty_est_all_ages"].replace('.', "0")
    latest_covid_cases.at[index, "median_household_income"] = row["median_household_income"].replace('.', "0")

In [55]:
latest_covid_cases.fillna(0, inplace=True)

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [56]:
latest_covid_cases["poverty_est_all_ages"] = latest_covid_cases["poverty_est_all_ages"].astype('int64')
latest_covid_cases["median_household_income"] = latest_covid_cases["median_household_income"].astype('int64')

C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\coled\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [57]:
latest_covid_cases.to_csv("../data/county-cases-latest-master.csv")

## Daily Cases

In [58]:
# Convert Dates to pd.Datetimes
us_covid_cases_df["date"] = pd.to_datetime(us_covid_cases_df["date"])

In [59]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20249,2021-03-05,Virginia,51,583060,9428
20250,2021-03-05,Washington,53,346733,5104
20251,2021-03-05,West Virginia,54,132964,2318
20252,2021-03-05,Wisconsin,55,620663,7099


In [60]:
# Create Daily New Cases Dataframe
daily_new_cases_df = us_covid_cases_df.groupby("date").sum()

In [61]:
daily_new_cases_df.reset_index(inplace=True)

In [62]:
# Add new Empty Columns for Daily New Cases and Deaths
daily_new_cases_df["daily_new_cases"] = ""
daily_new_cases_df["daily_new_deaths"] = ""

In [63]:
previous_day_cases = 0
previous_day_deaths = 0
    
for index, row in daily_new_cases_df.iterrows():
    
    # Cases
    new_cases = row["cases"] - previous_day_cases
    previous_day_cases = row["cases"]
    daily_new_cases_df.at[index, "daily_new_cases"] = new_cases
    
    # Deaths
    new_deaths = row["deaths"] - previous_day_deaths
    previous_day_deaths = row["deaths"]
    daily_new_cases_df.at[index, "daily_new_deaths"] = new_deaths

In [64]:
daily_new_cases_df["3_month_immunity_est"] = daily_new_cases_df.rolling(100)["daily_new_cases"].sum()

In [65]:
daily_new_cases_df["6_month_immunity_est"] = daily_new_cases_df.rolling(190)["daily_new_cases"].sum()

In [66]:
daily_new_cases_df = daily_new_cases_df.fillna(0)

In [67]:
daily_new_cases_df["date"] = daily_new_cases_df["date"].dt.strftime("%Y-%d-%m")

In [68]:
daily_new_cases_df

,date,fips,cases,deaths,daily_new_cases,daily_new_deaths,3_month_immunity_est,6_month_immunity_est
0,2020-21-01,53,1,0,1,0,0.0,0.0
1,2020-22-01,53,1,0,0,0,0.0,0.0
2,2020-23-01,53,1,0,0,0,0.0,0.0
3,2020-24-01,70,2,0,1,0,0.0,0.0
4,2020-25-01,76,3,0,1,0,0.0,0.0
...,...,...,...,...,...,...,...,...
405,2021-01-03,1762,28679618,514404,56672,1425,16507161.0,22965768.0
406,2021-02-03,1762,28737407,515710,57789,1306,16424112.0,22983153.0
407,2021-03-03,1762,28804121,518079,66714,2369,16311093.0,23010684.0
408,2021-04-03,1762,28871536,520028,67415,1949,16200605.0,23032780.0


In [69]:
daily_new_cases_df.to_csv("../data/state-daily-stats.csv")

### Set Up NYT Totals Master DF

In [70]:
us_covid_cases_df

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0
...,...,...,...,...,...
20249,2021-03-05,Virginia,51,583060,9428
20250,2021-03-05,Washington,53,346733,5104
20251,2021-03-05,West Virginia,54,132964,2318
20252,2021-03-05,Wisconsin,55,620663,7099


## Amazon RDS Updates

In [71]:
# Config Variables, and SQLalchemy
from config import endpoint, username, password
from sqlalchemy import create_engine

In [72]:
# Connect to AWS Database instance 
engine = create_engine(f'postgresql://uscovid:{password}@{endpoint}/us_covid_db')
connection = engine.connect()

In [73]:
# 50 States & D.C. Data
state_overview_master_df.to_sql('master_table', index=False, if_exists='replace', con=connection)

In [74]:
engine.execute('ALTER TABLE master_table ADD PRIMARY KEY (state);')

In [75]:
# Daily Cases Data
daily_new_cases_df.to_sql("daily_new_cases", index=False, if_exists='replace', con=connection)
engine.execute('ALTER TABLE daily_new_cases ADD PRIMARY KEY (date);')

In [76]:
# County Data
latest_covid_cases.to_sql("county_cases_latest", index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE county_cases_latest ADD PRIMARY KEY (index);')

In [77]:
# Raw NTY Data
us_covid_cases_df.to_sql('nyt_table', index=True, if_exists='replace', con=connection)
engine.execute('ALTER TABLE nyt_table ADD PRIMARY KEY (index);')